# GCN Depth train example

In [1]:
# !pip install -e ../package/
import os


from thesispack.models import *
from thesispack.methods import gen_nx_graphs, A2G, graphs_stats
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from itertools import product

In [ ]:
NewData = False
DataId = 0
saveData = False
path = '../data/gcn-depth/GraphsDataset_{}.npy'
if NewData:
    want = [1,0,0,0,0,1,1,1]
    examples = 300
    minN = 6
    maxN = 24

    X, A, Ahat, C = next(iter(gen_nx_graphs(examples, want, minN, maxN)))
    X = tf.cast(X,tf.float32)
    A = tf.cast(A,tf.float32)
    Ahat = tf.cast(Ahat,tf.float32)
    C = LabelBinarizer().fit_transform(C)
    C = tf.cast(C,tf.float32)
    if saveData:
        di = 0
        while True:
            if not os.path.exists(path.format(di)):
                np.save(path.format(di),
                    {
                        "A": A,
                        "A_hat": Ahat,
                        "X": X,
                        "C": C
                })
                break
            di += 1

else:
    A, Ahat, X, C = np.load(path.format(DataId), allow_pickle=True).tolist().values()
maxD, depth_dist, maxDs, edgesN = graphs_stats(A)
train = tf.data.Dataset.from_tensor_slices((X,Ahat,C)).batch(128)


In [ ]:
save_fig = True

vals = np.array(list(maxDs.values()))
vals = vals/np.sum(vals)

plt.figure(figsize=(16,4))
plt.bar(maxDs.keys(),vals)
plt.xticks(list(maxDs.keys()))
plt.xlabel(r'$Max \; Depths \; \#$', fontsize=20)
plt.ylabel(r'$Verteces \; \%$', fontsize=20)
if save_fig:
    plt.savefig('DataFigures/gcn-depth/{}.eps'.format('max-depth-dist'),format='eps')

In [ ]:
c = 0
cmax = C.shape[1]
for i in range(A.shape[0]):
    if c == tf.argmax(C[i]):
        c+=1
        g = A2G(A[i])
        plt.figure(figsize=(8,8))
        nx.draw(g)
    elif c == cmax:
        break

## GCN model Class Stracture

In [ ]:
class FC(tf.Module):
    def __init__(self, in_features,out_features,activation=None):
        super(FC,self).__init__(name="fc")

        self.weights = tf.Variable(
            tf.keras.initializers.GlorotUniform()(shape=[in_features, out_features]),
            name='weights'
        )
        self.bais = tf.Variable(
            tf.keras.initializers.GlorotUniform()(shape=[out_features]),
            name='bais'
        )
        self.activation = tf.keras.activations.get(activation)

    def __call__(self,inputs):
        x = tf.matmul(inputs,self.weights) + self.bais
        x = self.activation(x)
        return x

class GCNforDepth(tf.Module, Trainer):
    def __init__(self,nf0,nc,depth=1,nfi=64):
        tf.Module.__init__(self,name='my_gcn')
        Trainer.__init__(self,1e-2)
        self.depth = depth
        self.__status = [
            [0],
            [0],
            [1,2]
        ]
        self.score_mtr = MetricBase(self,
            [tf.keras.metrics.CategoricalAccuracy()],
            self.__status,
            [0]
        )
        self.cost_mtr = MetricBase(self,
            [tf.keras.metrics.CategoricalCrossentropy()],
            self.__status,
            [0],
            1
        )
        self.cost_loss = LossBase(self,
            # make custom cost
            [tf.keras.losses.CategoricalCrossentropy()],
            self.__status,
            [0]
        )
        depthi = '1'
        setattr(self, 'gcn{}'.format(depthi), GCN(nf0,nfi,'relu'))
        if self.depth > 1:
            l = 0.3/(self.depth-1)
        else:
            l = 1
        for d in range(1, self.depth):
            depthi = str(d+1)
            setattr(self, 'gcn{}'.format(depthi), GCN(nfi,nfi,'relu'))
            drop = l*d+0.1

        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = FC(nf0*nfi,256,'relu')
        self.out = FC(256,nc,'softmax')
    
    def __call__(self,inputs):
        depthi = '1'
        x = getattr(self, 'gcn{}'.format(depthi))(inputs[0],inputs[1])
        for d in range(1, self.depth):
            depthi = str(d+1)
            x = getattr(self, 'gcn{}'.format(depthi))(x,inputs[1])
        
        x = self.flatten(x)
        x = self.fc1(x)
        y = self.out(x)
        y = tuple([y])
        return y
    
    def set_knn_out(self, knn_out):
        pass


## Grid Training

In [ ]:
R = list(range(min(maxDs.keys()),max(maxDs.keys())+1))
E = list(range(100,400,100))

In [ ]:
di = 0
load_gcn_depth_results = True
if load_gcn_depth_results:
    NGScost = np.load("../data/gcn-depth/GScost_{}.npy".format(di),allow_pickle=True)[()]
    NGScost = NGScost["GScost"]

else:
    Nexpr = 10


    Nenum = list(product(list(range(Nexpr)), list(range(len(R))),list(range(len(E)))))
    NGS = list(product(list(range(Nexpr)), R, E))

    NGScost = np.zeros((Nexpr, len(R), len(E)))


    for (n, r, e), (i, j,w) in zip(NGS,Nenum):
        mygcn = GCNforDepth(Ahat.shape[1],4,r,8)
        mygcn.train(train,e,print_return_history=False)
        cost = mygcn.cost_mtr.metric_dataset(train)
        NGScost[i,j,w] = cost
        print('n exp: {}, r: {}, e: {}, cost: {}'.format(n,r,e,cost))



    while True:
        if not os.path.exists("GScost_{}.npy".format(di)):
            np.save("GScost_{}.npy".format(di),
                {
                    "GScost": NGScost
            })
            break
        di += 1

In [ ]:
cost_mean = NGScost.mean(axis=(0,2))
cost_std = NGScost.std(axis=(0,2))

In [ ]:
save_fig = True

cost_mean_min = np.min(cost_mean)
cost_mean_argmin = np.argmin(cost_mean)

plt.figure(figsize=(16,8))
plt.subplot(2,1,1)
plt.plot(R, cost_mean, 'o-')
plt.plot(R[cost_mean_argmin],cost_mean_min,'*r', markersize=12,label=r'$min \; cost$')
plt.legend(fontsize=12)
plt.xlabel(r'$GCN \; Depth \; \#$', fontsize=20)
plt.ylabel(r'$Mean \; Cost$', fontsize=20)
plt.fill_between(R, cost_mean - cost_std,
                     cost_mean + cost_std, alpha=0.1,
                     color="b")

if save_fig:
    plt.savefig('DataFigures/gcn-depth/{}.eps'.format('max-depth-grid-train'),format='eps')